In [1]:
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import numpy as np
import time
import random

In [2]:
def get_resource(url):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    while True:
        try:
            time.sleep(random.uniform(1, 5))
            r = requests.get(url, headers=headers)
            return r
        except:
            print("requests.get failed, so try again!")
            time.sleep(5)
            #r = requests.get(url, headers=headers, timeout=5)
            #return r
            continue
        break

In [3]:
def parse_html(r):
    if r.status_code == requests.codes.ok:
        r.encoding = 'utf8'
        soup = BeautifulSoup(r.text, 'lxml')
    else:
        print("Http請求錯誤: "+url)
        soup = None
    return soup

In [17]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
websites = []
for i in range(1,132): #4712筆, 131pages
    url=f'https://www.isave.com.tw/cars.aspx?pageindex={i}&sort_expr=car_yy&sort_dir=desc' #依照年份新到舊排列
    time.sleep(8)
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'lxml')
    car_info=soup.find_all(class_='listing') 
    for i in car_info:
        websites.append(i.a.get('href'))

In [19]:
len(set(websites))

4715

In [4]:
#pd_websites = pd.DataFrame(websites, columns=['isave_columns'])
#pd_websites.to_csv('isave_websites0706.csv', encoding='utf_8_sig')
websites = pd.read_csv('isave_websites0706.csv', index_col=0)
websites['isave_columns'][:3]

0    car.aspx?cid=195648
1    car.aspx?cid=195646
2    car.aspx?cid=193888
Name: isave_columns, dtype: object

In [17]:
#Second Step: Get the detail of the cars
#df_detail_basic=pd.DataFrame(columns=['車商', '車款', '車型', '年份', '縣市', '顏色', '排檔', '排氣量', '里程數'])
df_detail_basic=pd.DataFrame(columns=['dealer_name', 'car_brand', 'car_model', 'car_year', 'car_address', 'car_color', 'car_gear', 'car_cylinderVolume', 'car_mileage'])
#df_equipements=pd.DataFrame(columns=['皮椅','晶片系統','恆溫空調','VCD','ABS','CD','電動窗','中控鎖','電動後視鏡','天窗','霧燈','DVD','倒車顯影','衛星導航','TCS防滑','防盜','安全氣囊','定速','一手車','電動座椅','核桃木飾板','4WD','倒車雷達','行車電腦','總代理','MP3','柴油','ＨＩＤ頭燈','鋁圈'])
df_equipements=pd.DataFrame(columns=['皮椅','晶片系統','auto_AC','IVI_sys1','ABS','IVI_sys2','電動窗','中控鎖','電動後視鏡','Sunroof','霧燈','IVI_sys3','倒車顯影','衛星導航','TCS防滑','防盜','SRS','定速','一手車','electric_seat','核桃木飾板','4WD','倒車雷達','行車電腦','總代理','MP3','柴油','ＨＩＤ','鋁圈'])
df_others=pd.DataFrame(columns=[])


title_all = []
price = []
detail_basic_all = []
equipements_all = []
address_all = []
photo_all = []
certificated_all = []
id = []
for j in websites['isave_columns'][:1000]: #4717筆, 132pages
    try:
        url=f'https://www.isave.com.tw/{j}' 
        time.sleep(6)
        r=get_resource(url)
        soup=parse_html(r)

        #price
        tmp1=soup.find(class_='price').text.strip()
        price.append(tmp1)

        #title
        title = soup.find_all(class_="car-page-small-title")
        title2 = BeautifulSoup(str(title), 'html.parser').text
        title_all.append(title2) 

        #photo
        if soup.find(id="royalSlider-block", class_="royalSlider rsDefault image") is not None:
            car_photo = soup.find(id="royalSlider-block", class_="royalSlider rsDefault image")
            car_photo2 = BeautifulSoup(str(car_photo), 'html.parser').a
            photo_all.append('https://www.isave.com.tw'+car_photo2.get('href'))
        else:
            photo_all.append('')

        #df_detail_basic車商名...
        detail_basic = soup.find_all(class_='main-car-details')
        #區塊轉型為HTML字串，再給 bs4 進行一次宣告
        detail_basic_1 = BeautifulSoup(str(detail_basic), 'html.parser')
        detail_basic_2 = detail_basic_1.find_all(class_='option-content')
        tmp = []
        for z in detail_basic_2:
            tmp.append(z.text.strip())
        detail_basic_all.append(tmp)


        #df_equipements
        equipements = soup.find_all(class_='car-equipment')
        equipements2 = BeautifulSoup(str(equipements), 'html.parser')
        equipements3 = equipements2.find_all('li')
        tmp = []
        for k in equipements3:
            if k.img.get('src') == 'images/icon/isave_yes_icon.png':
                tmp.append('1')
            else:
                tmp.append('0')
        equipements_all.append(tmp)

        #seller_address
        if soup.find_all(class_='store-detail') is not None:
            address = soup.find_all(class_='store-detail')
            address2 = BeautifulSoup(str(address), 'html.parser')
            for i in address2.find_all('span'):
                if '地　　　　　址' in i.text:
                    address_all.append(i.text.strip().replace('地　　　　　址：　',''))
        else:
            address_all.append("")

        #certificated
        if soup.find_all(class_="car-tags") is not None:
            certificated = soup.find_all(class_="car-tags")
            certificated2 = BeautifulSoup(str(certificated), 'html.parser').div.text
            if '認證車' in certificated2:
                certificated_all.append('1')
            else:
                certificated_all.append('0')
        else:
            certificated_all.append('0')

        #id
        id.append(j.replace("car.aspx?cid=",""))
        
        
    except:
        continue
    

df1 = pd.DataFrame(detail_basic_all, columns=df_detail_basic.columns)
df2 = pd.DataFrame(equipements_all, columns=df_equipements.columns) 
df_detail_basic = df_detail_basic.append(df1, ignore_index=True) 
df_equipements = df_equipements.append(df2, ignore_index=True)
car_database = pd.concat([df_detail_basic, df_equipements,],axis =1)

requests.get failed, so try again!


In [18]:
#car_database.head()
car_database.shape

(1000, 38)

In [1]:
len(certificated_all) #這個只有995， 問題在這  無法弄在一起

NameError: name 'certificated_all' is not defined

In [33]:
df_detail_basic = df_detail_basic.append(df1, ignore_index=True)
print(df_detail_basic)
print(df_detail_basic.shape)

   dealer_name car_brand      car_model car_year car_address car_color  \
0         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
1         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
2         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
3         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
4         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
5      辛巴克有限公司      FORD           KUGA    2021年         台中市       白色系   
6         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
7         萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
8      辛巴克有限公司      FORD           KUGA    2021年         台中市       白色系   
9       廣一汽車商行    SUZUKI          JIMNY    2021年         嘉義縣       灰色系   
10        萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
11        萬通汽車    LUXGEN            URX    2021年         台中市       白色系   
12     辛巴克有限公司      FORD           KUG

In [24]:
df_detail_basic

,dealer_name,car_brand,car_model,car_year,car_address,car_color,car_gear,car_cylinderVolume,car_mileage
0,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
1,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
2,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
3,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
4,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
5,辛巴克有限公司,FORD,KUGA,2021年,台中市,白色系,手自排,1999CC,4275KM
6,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
7,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM
8,辛巴克有限公司,FORD,KUGA,2021年,台中市,白色系,手自排,1999CC,4275KM
9,廣一汽車商行,SUZUKI,JIMNY,2021年,嘉義縣,灰色系,自排,1462CC,48KM


In [19]:
websites['isave_columns'][:5]

0    car.aspx?cid=195648
1    car.aspx?cid=195646
2    car.aspx?cid=193888
3    car.aspx?cid=191384
4    car.aspx?cid=191312
Name: isave_columns, dtype: object

In [41]:
url

'https://www.isave.com.tw/car.aspx?cid=195570'

In [33]:
websites[:3]

['car.aspx?cid=195648', 'car.aspx?cid=195646', 'car.aspx?cid=193888']

In [127]:
pd.DataFrame(detail_basic_all, columns=df_detail_basic.columns)
#car_database.to_csv('car_isave_test.csv', encoding='utf_8_sig')

In [21]:
#car_database['adress'] = address_all
car_database['car_price'] = price
car_database['car_photo'] = photo_all
car_database['car_title'] = title_all
#car_database['verified'] = certificated_all
car_database['id'] = certificated_all
car_database.head()
#car_database.to_csv('car_isave_part1.csv', encoding='utf_8_sig')
#在這邊出錯

ValueError: Length of values (995) does not match length of index (1000)

In [8]:
len(address_all)

995

In [35]:
websites[:3]

['car.aspx?cid=195648', 'car.aspx?cid=195646', 'car.aspx?cid=193888']